# Deep Learning Inverse Kinematic

In diesem Notebook sollen Sie einen Einblick in das Thema Deep Learning mit Keras bekommen. Mit Hilfe von tiefen neuronalen Netzen werden Sie die Rückwärtstransformation für den YouBot von KUKA lösen. Dabei werden Sie lernen wie ein solches Netz mit Keras erzeugt wird und wie man es trainiert.

Hilfreiche Links:

- Inverse Recurrent Models - An Application Scenario for Many-Joint Robot Arm Control: https://uni-tuebingen.de/fakultaeten/mathematisch-naturwissenschaftliche-fakultaet/fachbereiche/informatik/lehrstuehle/kognitive-systeme/projects/inverse-kinematics-with-recurrent-neural-networks/
- Neural Network based Inverse Kinematics Solution for Trajectory Tracking of a Robotic Arm: https://www.sciencedirect.com/science/article/pii/S2212017313006361
- Applying Neural Network to Inverse Kinematic Problem for 6R Robot Manipulator with Offset Wrist: https://www.researchgate.net/publication/227064632_Applying_Neural_Network_to_Inverse_Kinematic_Problem_for_6R_Robot_Manipulator_with_Offset_Wrist
- A Neural Network Approach for Inverse Kinematic of a SCARA Manipulator: https://www.researchgate.net/publication/259198964_A_Neural_Network_Approach_for_Inverse_Kinematic_of_a_SCARA_Manipulator
- Keras API: https://keras.io/

## Idee

Wir möchten die Rückwärtstransformation für einen bestimmten Roboter, den YouBot, durchführen. Das bedeutet dass wir für einen von uns bestimmten Punkt im Raum die Gelenkwinkel bestimmen müssen die der Roboter einstellen muss, damit er diesen erreicht. Daraus ergeben sich auch Eingabe und Ausgabe des Netzes das wir erstellen, das Netz muss den TCP als Eingangsschicht erhalten und am Ende die Gelenkwinkelstellungen ausgeben.

## Vorbereitung der Daten

Einer der wichtigsten Aspekte beim supervised learning sind die Daten. Sie werden sehen, dass wir diese in unserem Fall ganz einfach erzeugen können. Da uns die Vorwärtstransformation bekannt ist können wir über diese für beliebig viele Gelenkwinkelstellungen den TCP errechnen lassen.

#### DH-Parametertabelle youBot

$\begin{array}{rr} \hline
\mathbf{Gelenk} &\mathbf{\theta} &\mathbf{d} &\mathbf{a} &\mathbf{\alpha} \\ \hline
\mathbf{1} &0        &0.075 &0.033 &-\pi / 2 \\ \hline
\mathbf{2} &-\pi / 2 &0     &0.155 &0 \\ \hline
\mathbf{3} &0        &0     &0.135 &0 \\ \hline
\mathbf{4} &\pi / 2  &0     &0     &\pi / 2 \\ \hline
\mathbf{5} &\pi / 2  &0.218 &0     &0 \\ \hline
\end{array}$

Für die Vorwärtstransformation werden die DH-Parameter des YouBots benötigt die wir im folgenden Codebeispiel als numpy Arrays anlegen.

```python
import numpy as np

dh_theta_values = np.array([0, -np.pi / 2, 0, np.pi / 2, np.pi / 2])
dh_alpha_values = np.array([-np.pi / 2, 0, 0, np.pi / 2, 0])
dh_a_values = np.array([0.033, 0.155, 0.135, 0, 0])
dh_d_values = np.array([0.075, 0, 0, 0, 0.218])
```

Wir programmieren nun eine Python-Funktion <b>dhIthFrame</b> zur Erstellung eines einzelnen DH-Frames. Als Parameter werden $\theta$, $d$, $a$ und $\alpha$ des entsprechenden Gelenkes angenommen.

```python
def dhIthFrame(theta, d, a, alpha):
    
    rot_theta = np.matrix([ [np.cos(theta), -np.sin(theta), 0, 0], 
                            [np.sin(theta), np.cos(theta), 0, 0], [0, 0, 1, 0], 
                            [0, 0, 0, 1] ])
    
    trans_d = np.matrix([ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, d], [0, 0, 0, 1] ])
    trans_a = np.matrix([ [1, 0, 0, a], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1] ])
    
    rot_alpha = np.matrix([ [1, 0, 0, 0], 
                            [0, np.cos(alpha), -np.sin(alpha), 0], 
                            [0, np.sin(alpha), np.cos(alpha), 0], [0, 0, 0, 1] ])
    
    dh_ith_frame = rot_theta * trans_d * trans_a * rot_alpha
    
    return dh_ith_frame
```

Dieses DH-Frame ist unsere Blaupause für die Forwärtstransformation am youBot. Wir definieren eine Python-Funktion <b>buildDhTcpFrame</b>, in welcher wir die Argumente eines solchen DH-Frames durch die DH-Parameterfür die Gelenkwinkel $a_{1}$ bis $a_{5}$ einsetzen und die resultierenden Frames zu einem Ergebnis-Frame akkumulieren.

```python
def buildDhTcpFrame(q_array):
    dh_frame = np.identity(4)
      
    for i in range(5):
        tmp_dh_ith = dhIthFrame(q_array[i] + dh_theta_values[i], 
                                dh_d_values[i], 
                                dh_a_values[i], 
                                dh_alpha_values[i])
        dh_frame = np.matmul(dh_frame, tmp_dh_ith)
    
    return dh_frame
```

Mit dieser Funktion haben wir nun die Möglichkeit TCP und Ausrichtung für übergebene Gelenkwinkelstellungen auszurechnen. Für die Erzeugung unseres Datensatzes brauchen wir nur noch die Anzahl an Gelenkwinkelstellungen.
Dazu beachten wir die Gelenkwinkelgrezen jedes Gelenks am YouBot.

$\begin{array}{rr} \hline
\mathbf{Gelenk} &{Grenzen°} &{Grenzen°} \\ \hline
\mathbf{A1} &-169°        &169°  \\ \hline
\mathbf{A2} &-65°         &90°  \\ \hline
\mathbf{A3} &-150°        &146°   \\ \hline
\mathbf{A4} &-102.5°      &102.5°   \\ \hline
\mathbf{A5} &-167.5°      &167.5° \\ \hline
\end{array}$

Es gibt viele denkbare Möglichkeiten diese Gelenkwinkel zu generieren, diese sollen nicht Teil dieses Notebooks sein. Um das Notebook nicht zu überfluten wurden einige Funktionen in das Skript utils abgelegt. Ein naiver Ansatz wäre beispielsweise das völlig zufällige generieren von Gelenkwinkeln. Ein kurzes Code-Snippet, wie sie es in der Klasse zu finden ist als Beispiel:
```python
for joint_range in joint_limits:
    joint_val = np.random.randint(joint_range[1], joint_range[0] + 1)
    degree_joint_pos.append(joint_val)
degree_joint_pos = np.asarray(degree_joint_pos)
radians = np.radians(degree_joint_pos)     
```

Die einzelnen Gelenkwinkelgrenzen werden in einer Liste abgespeichert und es wird ein zufälliger Wert innerhalb dieser Grenze gezogen und in eine weitere Liste gespeichert. Die Werte in Grad werden zum Schluss in Bogenmaß umgerechnet.

In [ ]:
import utils as u

datahandler = u.DataHandler(euler=False)

Zum Test erzeugen wir 100 Gelenkwinkelpositionen und die dazugehörigen TCP Frames.

In [ ]:
import time
import numpy as np

start = time.time()

joint_pos, tcp = datahandler.generate_data(1000)
end = time.time()
print("Berechnungszeit: %.2f Sekunden" %(end - start))

Die Dimension der generierten Daten kann man sich über die shape Funktion von numpy anzeigen lassen. Wir sehen dass wir für unser Netz eine Eingangsgröße von 12 Punkten haben, diese ergibt sich aus der Matrix des TCP, dabei wird die untere Reihe bewusst unterschlagen, da sie immer aus [0,0,0,1] besteht. Als Ausgang des Netzes haben wir die gewünschten 5 Gelenkwinkelpositionen.

In [ ]:
print("TCP Shape:", tcp.shape)
print("Joint Shape:", joint_pos.shape)

Nun können wir uns noch einige der TCP Frames anzeigen lassen und überprüfen ob sie richtig erzeugt wurden.

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from pytransform3d.rotations import *
%matplotlib notebook

ax = plot_basis(R=np.eye(3), ax_s=5)
f = 10
samples = np.random.randint(0, len(tcp), 10)
for i in samples:
    point = tcp[i][3::4] * f
    rot = np.asarray([tcp[i][0:3], tcp[i][4:7], tcp[i][8:11]])
#     RRT = np.dot(rot, rot.T)
#     print(RRT)
    plot_basis(ax, R=rot, p=point)

plt.show()


Wenn wir alles richtig gemacht haben ist unser Datengenerator fertig, er liefert uns Eingabe und Ausgabe für das Netz. Für das Trainieren mit Supervised Learning brauchen wir beides. Die Eingabe(TCP) wird durch das Netz propagiert und liefert eine Ausgabe(Gelenkwinkelstellung), diese wird mit den erwarteten Gelenkwinkelstellungen für den TCP verglichen und ein Fehler ausgerechnet. Anhand dieses Fehlers kann das Netz über den Backpropagation-Algorithmus seine Gewichte einstellen.

Um es dem Netz etwas einfacher zu machen wird die Rotationsmatrix aus dem TCP in die speziellen Eulerwinkel *raw*, *pitch* und *yaw* überführt. Dies wird ebenfalls durch die Datahandler Klasse übernommen, dafür muss lediglich der Parameter <code>euler=True</code> gesetzt werden. Möchte man wieder mit TCP Frames arbeiten setzt man diesen auf <code>False</code>

In [ ]:
datahandler.euler = True

Nun sollten wir uns im Vorfeld Gedanken darüber machen wie unser Netz aussehen sollte. Neben dem Input- und dem Output-Layer haben wir noch eine Anzahl an sogenannten Hidden-Layer. Die Anzahl der Units im Input- und Output-Layer sind klar definiert, aber im Hidden-Layer können beliebig viele Neuronen mit diesen Schichten verbunden werden. 

Wir benutzen Keras um das Netz aufzubauen, da es sich um ein einfaches Feed Forward Netz handeln soll benutzen wir die **Sequential()** Klasse von Keras um das Modell zu erstellen. Diesem Modell können nun Layer hinzugefügt werden analog zu unseren vorherigen Überlegungen. Ein Fully Connected Layer kann über die Klasse **Dense()** erstellt werden, wir definieren hier beispielsweise die Inputdimension(*input_dim*), die Ausgangsdimension(*units*) und die Aktivierungsfunktion. Dieses Layer kann über <code>model.add</code> dem Model hinzugefügt werden. So kann man das Netz sehr einfach von vorne nach hinten aufbauen.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2

model = Sequential()
    
layer = Dense(units=64, 
          input_dim=6,
          kernel_initializer='random_normal',
          use_bias=False,
          activation='relu')

model.add(layer)

hidden_layers = 4

for i in range(hidden_layers):
    layer = Dense(units=64, 
                  kernel_initializer='random_normal',
                  use_bias=True,
                  activation='relu') #,kernel_regularizer=l1_l2(l1=0.01, l2=0.01)
    model.add(layer)
    
layer = Dense(units=5, 
              kernel_initializer='random_normal',
              use_bias=False,
              activation='linear')
model.add(layer)


Nun haben wir unsere Traningsdaten und unser Netz das trainiert werden soll. Bevor wir dieses trainieren können muss der Lernprozess konfiguriert werden. Dies geschieht über die <code>compile()</code> Funktion des Modells. Hier müssen noch der Optimierer und die Loss-Funktion übergeben werden.
Die Lossfunktion ermittelt wie der Fehler am Ausgang des Netzes berechnet wird, welcher minimiert werden soll. Der Optimierer ist für das Einstellen der Gewichte im Netz zuständig.
In diesem Fall nehmen wir den Adam Optimierer von Keras und den *mean squared error* als Loss Funktion.

In [ ]:
from tensorflow.keras.optimizers import Adam, Adadelta
# For a mean squared error regression problem
adam=Adam(learning_rate=1.0e-4)
adadelta = Adadelta()

model.compile(optimizer=adam,loss='mse')


Man kann sich das kompilierte Modell in einer Tabelle anzeigen lassen.

In [ ]:
model.summary()

Tensorflow gibt es in einer CPU und einer GPU Variante. Wenn man sich nicht sicher ist welche gerade in diesem Notebook läuft kann man es sich vorher nochmal ausgeben lassen.

In [ ]:
import tensorflow as tf
#print("All your devices: ", tf.config.experimental.list_physical_devices(device_type=None), "\n")
print("GPUs available: ", tf.config.list_physical_devices('GPU'), "\n") # True/False
print("Tensorflow Version: ", tf.__version__)

Nun können wir endlich unser Modell trainieren. Vorher erzeugen wir uns noch einen neuen Datensatz. 100 Beispiele werden nicht reichen um ein zufriedenstellendes Ergebnis zu bekommen. Es sollten pro Durchlauf mindestens 100000 Datensätze erzeugt werden.
Um das Training zu starten rufen wir die <code>fit</code> Funktion des Modells auf. Dieser übergeben wir die Parameter ***x*** Input Daten, ***y*** Ziel Daten, ***epochs*** Anzahl an Durchläufen durch den gesamten Datensatz, ***batch_size*** Anzahl an Daten pro Update der Gewichte, ***validation_split*** prozentuale Aufteilung zwischen Trainingsdaten und Validierungsdaten.

Während des Trainings werden wir die Ausgabe des Netzes beobachten können, wir sehen den<code>loss</code>der aktuellen Epoche und ihren<code>val_loss</code>. Dies ist der ermittelte *mean squared error* der Lossfunktion für Trainingsdaten und Validierungsdaten. Dieser Fehler sollte eine abnehmende Tendenz aufweisen, dabei ist zu beachten dass der<code>val_loss</code> aussagekräftiger ist. Wenn der<code>loss</code> der Trainingsdaten wesentlich kleiner ist als der<code>val_loss</code>weist dies auf *Overfitting* hin, was bedeutet dass das Netz nicht generalisiert, sondern die Trainingsdaten "auswendig lernt".

In [ ]:
for i in range(5):
    jpos, tpos = datahandler.generate(100000)
    if (i%1 == 0):
        print(i)
    model.fit(x=tpos, y=jpos, epochs=3, batch_size=32, validation_split=0.1)

So trainierte Modelle können über die Funktionen <code>save()</code> und <code>load_weights()</code> gespeichert und geladen werden. Achten Sie darauf, dass das Modell vorher initialisiert sein muss, falls Sie an der Struktur des Netzes etwas ändern wird die Funktion <code>load_weights()</code> das Modell nicht mehr laden können.
```python
model.save('model_IK.h5')
model.load_weights('model_IK.h5')
```

In [ ]:
model.save('model_IK.h5')

In [ ]:
model.load_weights('model_IK.h5')

Nun wollen wir überprüfen ob unser Netz auch auf neuen Daten funktioniert. Dafür generieren wir uns wieder einige Daten. Die so generierten TCPs dienen wieder als Eingabe in unser Modell. Über die <code>predict()</code> Funktion können wir unsere Daten durch das Netz propagieren um eine Ausgabe zu erhalten. Was bislang verschwiegen wurde ist, dass die Daten bevor sie in das Netz gespeist wurden normalisiert waren, diese Normalisierung muss rückgängig gemacht werden, bevor wir weiter mit den Daten arbeiten können.

In [ ]:
gen_pos, gen_tcp = datahandler.generate(500)

pred = model.predict(gen_tcp)
gen_pos = datahandler.denormalize(gen_pos)
pred = datahandler.denormalize(pred)

Zur Vorbereitung für die nächste Aufgabe definieren wir schon einmal eine Funktion <code>custom_error()</code>. Diese soll später dafür dienen den *mean squared error* als Loss Funktion abzulösen. Wir können mit dieser Funktion aber auch schon die Genauigkeit unseres Netzes errechnen.

Dazu wird für jede Ausgabe des Netzes noch einmal die Vorwärtstransformation durchgeführt um den TCP zu errechnen. Der korrekte TCP gilt hierbei als Vergleichswert. Dann werden die Punktkoordinaten und die Drehmatrix der beiden Frames isoliert. Für die Drehmatrix berechnen wir den *mean squared error* und für die beiden Punkte wird der euklidische Abstand berechnet. Anschließend werden beide Werte mit unterschiedlicher Gewichtung addiert um auf den neuen Fehler zu kommen.

In [ ]:
def custom_error(y_gt, angles_pred, euler=True):

    tcp_out = u.buildDhTcpFrame(angles_pred)
    tcp_out = np.asarray(tcp_out.flatten())
    tcp_out = tcp_out[0 :12]
    tcp_out = datahandler.norm(np.expand_dims(tcp_out, axis=0))  
    tcp_out = tcp_out.flatten()
    
    if not euler:
        xyz_gt = y_gt[3::4]
        xyz_pred = tcp_out[3::4]
        rot_gt = np.asarray([y_gt[0:3], y_gt[4:7], y_gt[8:11]]).flatten()
        rot_pred = np.asarray([tcp_out[0:3], tcp_out[4:7], tcp_out[8:11]]).flatten()
    else:
        xyz_gt = y_gt[0:3]
        xyz_pred = tcp_out[3::4]
        rot_gt = y_gt[3:6]
        rot_pred = np.asarray([tcp_out[0:3], tcp_out[4:7], tcp_out[8:11]]).flatten()
        roll, pitch, yaw = datahandler.get_roll_pitch_yaw(rot_pred)
        rot_pred = np.asarray([roll, pitch, yaw])
        
    mse = np.mean(np.sqrt((rot_gt - rot_pred) ** 2))
    dist = np.linalg.norm(xyz_gt - xyz_pred)
    
    err = 0.1 * mse + 0.9 * dist

    return err

In [ ]:
def mean_error(y_gt, angles_pred):
    return np.mean(np.sqrt((y_gt - angles_pred) ** 2))

Wir iterieren über die Länge des Ausgangsvektors und kummulieren den Fehler auf, anschließend teilen wir ihn durch die Länge.

In [ ]:
import numpy as np
cus_err = 0
mse = 0

for i in range(len(pred)):
    loss = custom_error(gen_tcp[i], pred[i])
    cus_err += loss
    loss = mean_error(gen_pos[i], pred[i])
    mse += loss

print("median custom error: ", cus_err/len(pred))
print("median mse: ", mse/len(pred))

Wir können uns nun für einzelne Werte die Roboterstellungen für *Ground Truth* und *Prediction* der Gelenkwinkelstellungen anzeigen lassen. Orange bildet hierbei die *Ground Truth* ab und grün die *Prediction*, so bekommen wir eine grapfische Repräsentation von dem was unser Netz gelernt hat.

In [ ]:
%matplotlib notebook

drawer = u.Drawer()
t = 200 #Nummer des Datensatzes
drawer.draw_robot(gen_pos[t], pred[t])
#for t in range(1,500,50):
#    print (t)
#print("Custom Loss: ", custom_error(gen_tcp[t], pred[t]))
#print("Mean Squared Error: ", mean_error(gen_pos[t], pred[t]))

Um einen noch besseren Eindruck zu bekommen ob unser Netz gut arbeitet können wir Gelenkwinkelpositionen generieren die eine bestimmte Trajektorie abfahren. Wir lassen uns die Trajektorie mit matplotlib ausgeben, wenden die Vorwärtstransformation auf unsere so generierten Gelenkwinkel an um den TCP zu bekommen. Diesen geben wir in unser trainiertes Netz und für den so generierten Ausgangsvektor der Gelenkwinkel lassen wir erneut die Trajektorie abfahren.

Als erstes generieren wir einfachhalber einen Gelenkwinkelvektor als numpy array.

In [ ]:
start = np.array([169, 0, -20, -30, 0])

Ausgehend von unserem generierten Startwert können nun iterativ ausgewählte Werte des Vektors verändert werden um unterschiedliche Trajektorien abzufahren. Dabei sollte darauf geachtet werden die Gelenkwinkelgrenzen des YouBots einzuhalten.

In [ ]:
arr = []
arr.append(start)
for i in range(26):
    start = np.copy(start)
    if(start[0]-13)>=datahandler.a1[1]:
        start[0] += -13
    arr.append(start)

arr = np.radians(np.asarray(arr))

Der so erzeugte Gelenkwinkelvektor kann der Funktion <code>draw_trajectory()</code> der Drawer Klasse übergeben werden um mit matplotlib eine animierte Trajektorie anzuzeigen.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pytransform3d.rotations import *
import matplotlib.animation as animation

%matplotlib notebook

drawer = u.Drawer()

drawer.draw_trajectory_robot(arr)

Wir erzeugen aus dem Gelenkwinkelvektor erneut die dazugehörigen TCP Frames und normalisieren sie anschließend, damit sie in das Netz gespeist werden können. Wir lassen uns über das Netz die berechneteten Gelenkwinkelstellungen ausgeben und denormalisieren diese.

In [ ]:
arr_tcp = datahandler.calc_xyz_euler(arr)
arr_tcp = datahandler.normeuler(arr_tcp)

shape = model.predict(arr_tcp)
shape = datahandler.denormalize(shape)

Zum Schluss können wir die so erzeugen Gelenkwinkelpositionen erneut als Trajektorie abfahren lassen und bekommen so ein visuellen Vergleich zwischen Soll und Ist.

In [ ]:
%matplotlib notebook
drawer2 = u.Drawer()

drawer2.draw_trajectory_robot(shape)